## Simulator

This notebook is used to generate simulated traffic of a nominal state and a broken slave.

In [ ]:
OUTPUT_DATA_PATH="../data/can_generated/"

In [495]:
from datetime import datetime,timedelta

def current_timestamp():
    return int(datetime.now().timestamp()*1000000)

def timestamp_to_datetime(ts):
    return datetime.fromtimestamp(ts/1000000) +timedelta(microseconds=ts % 1000000)

In [496]:
class Event(object):
    def __init__(self, timestamp: int):
        self.timestamp = timestamp
        self.sim = None

    def __lt__(self, other):
        return self.timestamp < other.timestamp
    
    def set_sim(self,sim):
        self.sim = sim
    
    def get_sim(self):
        return sim
        
    def entrypoint(self):
        pass

In [497]:
import heapq
 
class EventQueue:
    def __init__(self):
        self._queue = []
        self._index = 0
 
    def push(self, ev:Event):
        heapq.heappush(self._queue, (ev.timestamp, self._index, ev))
        self._index += 1
 
    def pop(self):
        return heapq.heappop(self._queue)[-1]
    
    def size(self):
        return len(self._queue)
        

In [498]:
import heapq
    
class Simulator:
    def __init__(self):
        self.q = EventQueue()
        self.ts = 0
        #self.q.push(MessageEvent(0,'Simulation started'))
        self.keep_running = True
        
    def post_event(self, ev):
        ev.set_sim(self)
        self.q.push(ev)
        
    def stop(self):
        self.q = EventQueue()
        self.keep_running = False
        
    def get_time(self):
        return self.ts
    
    def run_for(self,dt):
        self.ts = current_timestamp()
        self.post_event(SimulatorStopEvent(self.ts+dt))
        print("Running for %d microseconds" % dt )
        print("Simulation starting at time", timestamp_to_datetime(self.ts) )
        while self.keep_running:
            while self.q.size() > 0:
                ev=self.q.pop()
                if ev.timestamp is 0:
                    ev.timestamp = self.ts
                elif ev.timestamp < self.ts:
                    raise ValueError("Event timestamp is old %d" % ev.timestamp)
                self.ts = ev.timestamp
                ev.entrypoint()       
        print("Simulation ended at time", timestamp_to_datetime(self.ts) )

In [499]:
class SimulatorMessageEvent(Event):

    def __init__(self, timestamp, message):
        Event.__init__(self,timestamp)
        self.message = message
        
    def entrypoint(self):
        print(self.message)
        pass

In [500]:
class SimulatorStopEvent(Event):

    def __init__(self, timestamp):
        Event.__init__(self,timestamp)
        
    def entrypoint(self):
        self.get_sim().stop()
        pass

### CAN Models

In [501]:
import numpy as np

def encode_canid(priority,function,subfunction, node_id):
    return (priority<<24)|(function<<16)|(subfunction<<8)|node_id

def decode_canid(canid):
    return (canid>>24)&0xFF, (canid>>16)&0xFF, (canid>>8)&0xFF,canid&0xFF


class CANMessage:
    def __init__(self, canid, payload):
        self.canid = canid
        self.payload = payload
        
    def __str__(self):
        tmp = "0x%02x,0x%02x,0x%02x,0x%02x,%d," % (*decode_canid(self.canid),len(self.payload)) 
        for x in self.payload:
            tmp += "0x%02X," % x
        tmp += "0x00," *(8-len(self.payload))
        return tmp

class CANReceiveMessageEvent(Event):

    def __init__(self, timestamp, msg, bus):
        Event.__init__(self,timestamp)
        self.msg = msg
        self.bus = bus
        
    def entrypoint(self):        
        self.bus.handle_msg(self.timestamp,self.msg)

class CANSendMessageEvent(Event):
    TRANSMISSION_TIME_IN_US = 10

    def __init__(self, timestamp, msg, bus):
        Event.__init__(self,timestamp)
        self.msg=msg
        self.bus = bus
        
    def entrypoint(self):
        self.get_sim().post_event(
            CANReceiveMessageEvent(self.timestamp+self.TRANSMISSION_TIME_IN_US,self.msg, self.bus))
        
class CANMessageHandler:
    def __init__(self):
        pass
    def handle_msg(self,timestamp,msg):
        pass
        
        
class CANBusModel(CANMessageHandler):
    def __init__(self,sim):
        CANMessageHandler.__init__(self)
        self.handlers = []
        self.sim = sim
    
    def connect(self, handler):
        self.handlers.append(handler)
        
    def transmit(self,msg,jitter=0):
        sim.post_event(
            CANSendMessageEvent(
                self.sim.get_time()+jitter, msg, self)
        )
        
    
    def handle_msg(self,timestamp,msg):
        for h in self.handlers:
            h.handle_msg(timestamp,msg)           

#### Some monitors

In [502]:
class CANMessagePrinter(CANMessageHandler):
    def __init__(self):
        CANMessageHandler.__init__(self)
        pass
    
    def handle_msg(self,timestamp,msg):
        print("%d,MSG_RX,%s" % (timestamp,msg))

In [503]:
class CANMessageCSVLogger(CANMessageHandler):
    def __init__(self):
        CANMessageHandler.__init__(self)        
        pass
    
    def open_file(self,filename):
        self.fp = open(filename, 'w')
        self.fp.write("timestamp,event_type,priority,function,subfunction,node_id,len,d0,d1,d2,d3,d4,d5,d6,d7,\n")
        
    def close_file(self):
        self.fp.close()
    
    def handle_msg(self,timestamp,msg):
        self.fp.write("%d,MSG_RX,%s\n" % (timestamp,msg))

### CAN Master Sync Node

In [504]:
MASTER_SYNC_FUNCTION = 0xF0
MASTER_SYNC_SUBFUNCTION = 0xF1
MASTER_SYNC_NODE = 0x34
PTU_INTERVAL_IN_US = 50000
SYN_MSG_JITTER_IN_US = 3


class CANMasterSyncEvent(Event):
    
    def __init__(self, timestamp, bus):
        Event.__init__(self,timestamp)
        self.bus = bus
        self.current_ptu = 0
        
    def entrypoint(self):
        sim.post_event(
            CANSendMessageEvent(self.timestamp,
                CANMessage(
                    encode_canid(0,MASTER_SYNC_FUNCTION,MASTER_SYNC_SUBFUNCTION,MASTER_SYNC_NODE),
                    [ 0, self.current_ptu ]
                )
                , self.bus      
            )
        )
        self.timestamp += PTU_INTERVAL_IN_US + np.random.randint(-SYN_MSG_JITTER_IN_US, SYN_MSG_JITTER_IN_US)
        self.current_ptu = self.current_ptu + 1
        if self.current_ptu == 20:
            self.current_ptu = 0
        self.sim.post_event(self)
        
class CANMasterSyncNode:
    
    def __init__(self,sim, bus):
        self.sim = sim        
        self.bus = bus
        
    def start(self):
        sim.post_event( CANMasterSyncEvent(0, self.bus) )

### CAN Slave Node

In [518]:
SLAVE1_FUNCTION = 0xF4
SLAVE1_TM1_SUBFUNCTION = 0xD0
SLAVE1_NODE = 0x10
SLAVE_MSG_JITTER_IN_US = 3


class CANSlaveNode:
    
    def __init__(self,sim, bus, success_threeshold=0):
        self.sim = sim        
        self.bus = bus
        self.success_threeshold = success_threeshold
        self.fn_table = {
            # Function
            MASTER_SYNC_FUNCTION: {
                MASTER_SYNC_SUBFUNCTION: self.handle_sync
            }
        }
        
    def set_success_threeshold(self,success_threeshold):
        self.success_threeshold = success_threeshold
        
    def handle_sync(self,timestamp,msg):
        success = np.random.randint(0,100)
        if success >= self.success_threeshold:
            self.bus.transmit(
                CANMessage(
                    encode_canid(1,SLAVE1_FUNCTION,SLAVE1_TM1_SUBFUNCTION,SLAVE1_NODE),
                    #[0xFE,0xDE,0xBE,0xBE,0xCA,0xFE]
                    [0x11,0x2E,0x4E]
                 ),
                 np.random.randint(0, SLAVE_MSG_JITTER_IN_US)
            )
        
    def handle_msg(self,timestamp,msg):
        priority,function,subfunction, node_id = decode_canid(msg.canid)        
        if function in self.fn_table:
            if subfunction in self.fn_table[function]:
                self.fn_table[function][subfunction](timestamp,msg)
           

### Simulation

In [506]:
TOTAL_SIMULATION_TIME_IN_SECONDS = 10
CSV_FILENAME_GOOD = OUTPUT_DATA_PATH+"%d_second_master_and_slave.csv" % TOTAL_SIMULATION_TIME_IN_SECONDS
CSV_FILENAME_BAD = OUTPUT_DATA_PATH+"%d_second_master_and_slave_broken.csv" % TOTAL_SIMULATION_TIME_IN_SECONDS

In [507]:
sim = Simulator()

# Define CAN Network
canbus = CANBusModel(sim)

can_master = CANMasterSyncNode(sim,canbus)
can_slave = CANSlaveNode(sim,canbus)
canbus.connect( can_slave )

# Monitors
csv_logger =  CANMessageCSVLogger()
csv_logger.open_file(CSV_FILENAME_GOOD)
canbus.connect( csv_logger )

# Simulate
can_master.start()
sim.run_for( TOTAL_SIMULATION_TIME_IN_SECONDS *1000000 )
csv_logger.close_file()

Running for 10000000 microseconds
Simulation starting at time 2020-03-16 00:19:11.666382
Simulation ended at time 2020-03-16 00:19:21.666382


In [519]:
sim = Simulator()

# Define CAN Network
canbus = CANBusModel(sim)

can_master = CANMasterSyncNode(sim,canbus)
can_slave = CANSlaveNode(sim,canbus,50)
canbus.connect( can_slave )

# Monitors
csv_logger =  CANMessageCSVLogger()
csv_logger.open_file(CSV_FILENAME_BAD)
canbus.connect( csv_logger )

# Simulate
can_master.start()
sim.run_for( TOTAL_SIMULATION_TIME_IN_SECONDS *1000000 )
csv_logger.close_file()

Running for 10000000 microseconds
Simulation starting at time 2020-03-16 00:27:25.593772
Simulation ended at time 2020-03-16 00:27:35.593772


In [520]:
import pandas as pd

In [521]:
pd.read_csv(CSV_FILENAME_BAD)

timestamp event_type priority function subfunction node_id  len  \
0    1584318444796896     MSG_RX     0x00     0xf0        0xf1    0x34    2   
1    1584318444796906     MSG_RX     0x01     0xf4        0xd0    0x10    3   
2    1584318444846897     MSG_RX     0x00     0xf0        0xf1    0x34    2   
3    1584318444896894     MSG_RX     0x00     0xf0        0xf1    0x34    2   
4    1584318444896905     MSG_RX     0x01     0xf4        0xd0    0x10    3   
5    1584318444946896     MSG_RX     0x00     0xf0        0xf1    0x34    2   
6    1584318444996894     MSG_RX     0x00     0xf0        0xf1    0x34    2   
7    1584318445046893     MSG_RX     0x00     0xf0        0xf1    0x34    2   
8    1584318445096891     MSG_RX     0x00     0xf0        0xf1    0x34    2   
9    1584318445146890     MSG_RX     0x00     0xf0        0xf1    0x34    2   
10   1584318445146900     MSG_RX     0x01     0xf4        0xd0    0x10    3   
11   1584318445196888     MSG_RX     0x00     0xf0        0xf1    0x34    2   
12   1584318445246885     MSG_RX     0x00     0xf0        0xf1    0x34    2   
13   1584318445246896     MSG_RX     0x01     0xf4        0xd0    0x10    3   
14   1584318445296887     MSG_RX     0x00     0xf0        0xf1    0x34    2   
15   1584318445296897     MSG_RX     0x01     0xf4        0xd0    0x10    3   
16   1584318445346884     MSG_RX     0x00     0xf0        0xf1    0x34    2   
17   1584318445346895     MSG_RX     0x01     0xf4        0xd0    0x10    3   
18   1584318445396882     MSG_RX     0x00     0xf0        0xf1    0x34    2   
19   1584318445446880     MSG_RX     0x00     0xf0        0xf1    0x34    2   
20   1584318445446892     MSG_RX     0x01     0xf4        0xd0    0x10    3   
21   1584318445496878     MSG_RX     0x00     0xf0        0xf1    0x34    2   
22   1584318445496889     MSG_RX     0x01     0xf4        0xd0    0x10    3   
23   1584318445546875     MSG_RX     0x00     0xf0        0xf1    0x34    2   
24   1584318445596872     MSG_RX     0x00     0xf0        0xf1    0x34    2   
25   1584318445646873     MSG_RX     0x00     0xf0        0xf1    0x34    2   
26   1584318445696875     MSG_RX     0x00     0xf0        0xf1    0x34    2   
27   1584318445746872     MSG_RX     0x00     0xf0        0xf1    0x34    2   
28   1584318445746884     MSG_RX     0x01     0xf4        0xd0    0x10    3   
29   1584318445796871     MSG_RX     0x00     0xf0        0xf1    0x34    2   
..                ...        ...      ...      ...         ...     ...  ...   
279  1584318453846849     MSG_RX     0x01     0xf4        0xd0    0x10    3   
280  1584318453896840     MSG_RX     0x00     0xf0        0xf1    0x34    2   
281  1584318453896852     MSG_RX     0x01     0xf4        0xd0    0x10    3   
282  1584318453946840     MSG_RX     0x00     0xf0        0xf1    0x34    2   
283  1584318453996838     MSG_RX     0x00     0xf0        0xf1    0x34    2   
284  1584318453996848     MSG_RX     0x01     0xf4        0xd0    0x10    3   
285  1584318454046838     MSG_RX     0x00     0xf0        0xf1    0x34    2   
286  1584318454096838     MSG_RX     0x00     0xf0        0xf1    0x34    2   
287  1584318454146837     MSG_RX     0x00     0xf0        0xf1    0x34    2   
288  1584318454146847     MSG_RX     0x01     0xf4        0xd0    0x10    3   
289  1584318454196834     MSG_RX     0x00     0xf0        0xf1    0x34    2   
290  1584318454246836     MSG_RX     0x00     0xf0        0xf1    0x34    2   
291  1584318454246846     MSG_RX     0x01     0xf4        0xd0    0x10    3   
292  1584318454296838     MSG_RX     0x00     0xf0        0xf1    0x34    2   
293  1584318454296848     MSG_RX     0x01     0xf4        0xd0    0x10    3   
294  1584318454346836     MSG_RX     0x00     0xf0        0xf1    0x34    2   
295  1584318454396833     MSG_RX     0x00     0xf0        0xf1    0x34    2   
296  1584318454446830     MSG_RX     0x00     0xf0        0xf1    0x34    2   
297  1584318454496832     MSG_RX     0x00     0xf0        0xf1